In [24]:
%pwd

'C:\\Users\\zeeshan.hameed\\Desktop\\assignment1\\assign1TSFD'

In [25]:
import os
os.chdir(r'C:\Users\zeeshan.hameed\Desktop\assignment1\assign1TSFD')

In [26]:
%pwd

'C:\\Users\\zeeshan.hameed\\Desktop\\assignment1\\assign1TSFD'

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareModelConfig:
    root_dir : Path
    update_base_model_path :Path
    params_look_back : int


In [28]:
from timeseries.constants import *
from timeseries.utils.common import read_yaml,create_directories


class ConfigrationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def prepare_model_config(self)->PrepareModelConfig:
        config=self.config.prepare_base_model
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareModelConfig(root_dir=Path(config.root_dir),update_base_model_path=Path(config.updated_base_model_path),
                                                           params_look_back=self.params.LOOK_BACK)
        return prepare_base_model_config





In [29]:
import tensorflow as tf
from keras.models import Sequential
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import Dense,Dropout
from keras.layers import LSTM



class TSmodel:
    def __init__(self,config :PrepareModelConfig ):
        self.config = config
    
    def get_model(self):
        look_back = self.config.params_look_back
        model = Sequential()
        model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(look_back, 1)))
        model.add(LSTM(50, activation='relu', return_sequences=True))
        model.add(LSTM(50, activation='sigmoid', return_sequences=False))
        model.add(Dense(50))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.summary()
        return model
    def save(self,model : keras.Model):
        path = self.config.update_base_model_path
        model.save_weights(path)


In [30]:
config=ConfigrationManager()
prepare_model_config=config.prepare_model_config()
tsmodel=TSmodel(config=prepare_model_config)
tsmodel.get_model()

[2024-02-02 09:07:56,870: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-02 09:07:56,873: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-02 09:07:56,875: INFO: common: created directory at: artifacts]
[2024-02-02 09:07:56,876: INFO: common: created directory at: artifacts/prepare_base_model]
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 50)            10400     
                                                                 
 lstm_7 (LSTM)               (None, 30, 50)            20200     
                                                                 
 lstm_8 (LSTM)               (None, 50)                20200     
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                            